We use the newely tarined embeddings for word2vec transformation

There is a change in preprocessing in this notebook, we removed all the numbers from tweets, which helped in training more robust word2vec embeddings

Analysis on scraped dataset

### Import

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pickle
import sys
import nltk
from nltk.stem.porter import *

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cf_data_1 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/hatespeech_NAACL_SRW.csv',encoding = "ISO-8859-1")
cf_data_2 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/hatespeech_NLP+CSS.csv')

cf_data_3 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_2/labeled_data.csv',encoding = "ISO-8859-1")
## this is the scraped data

cf_data_3.rename({'Unnamed: 0':'ID','tweet':'Tweets'},axis=1,inplace=True)

labels_1 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/NAACL_SRW_2016.csv',header=None,names=['ID','class'])
labels_2 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/NLP+CSS_2016.csv',sep='\s')

labels_2.rename({'TweetID':'ID','Expert':'class'},axis=1,inplace=True)

cf_data_1.rename({'Unnamed: 0':'index_col'},axis=1,inplace=True)
cf_data_2.rename({'Unnamed: 0':'index_col'},axis=1,inplace=True)

### Function for merging

In [ ]:
def label_merging(data, labels):
    labels['ID'] = labels['ID'].astype(int)
    print(labels['ID'].nunique())
    print('Null IDs in data 1 = ' ,data['ID'].isna().sum())
    
    data['ID'].fillna(0,inplace=True)
    data['ID'] = data['ID'].astype(int)
    
    print('data shape ='  ,data.shape)
    print('IDs common in data and labels =',sum(data['ID'].isin(labels['ID'])))
    
    train = data.merge(labels, on='ID',how='inner')#['class'].isna().sum()
    return train

In [ ]:
train_1 = label_merging(cf_data_1,labels_1)

train_2 = label_merging(cf_data_2, labels_2)

train_3 = cf_data_3.copy()

In [ ]:
t1 = train_1[['ID','Tweets','class']]
t2 = train_2[['ID','Tweets','class']]
t3 = train_3[['ID','Tweets','class']]
merged = pd.concat([t1,t2,t3],axis=0).reset_index(drop=True)

### Target Analysis

### Basic Preprocessing

In [ ]:
train = merged.copy()

train.rename(columns={'Tweets':'tweet'},inplace=True)
train['tweet'] = train['tweet'].astype(str)

Every word followed by @ is some twitter ID of an user, which shouldn't be considered in our analysis, so lets do the stemming, where we remove @ alonwith the word followed by it

In [ ]:
train['tweet'] = train['tweet'].apply(lambda x:' '.join(i for i in [a for a in x.split() if a.find('@')==-1]))
train['tweet'] = train['tweet'].apply(lambda x:' '.join(i for i in [a for a in x.split() if a.find('http')==-1]))
train['tweet'] = train['tweet'].apply(lambda x:''.join([i for i in x if not i.isdigit()]))
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train['tweet'] = train['tweet'].str.replace('[^\w\s]','')

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

remove_word = ['rt','mkr','im']
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in remove_word))

Doesnt really make sense to remove rare words, i.e. the words with count 1. Because we might lose hateful words this way

In [ ]:
from textblob import TextBlob
nltk.download('punkt')

from textblob import Word
nltk.download('wordnet')
train['tweet'] = train['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train['tweet'].head()

### Target creation

In [ ]:
train['class'].unique()#.isna().sum()

In [ ]:
train['class'].replace(['racism', 'sexism',0, 1, 'both', 'none', 'neither',2],['hate','hate','hate','hate','hate','null','null','null'],inplace=True)
train['class'].value_counts()

In [ ]:
train['class'].replace(['null','hate'],[0,1],inplace=True)

### Removing duplicates

In [ ]:
# hate_text = train[train['class']==1]['tweet']
# null_text = train[train['class']==0]['tweet']

In [ ]:
# hate_text.to_csv(r'hate_speech.txt', header=None, index=None, sep=' ')

In [ ]:
# null_text.to_csv(r'null_speech.txt', header=None, index=None, sep=' ')

In [ ]:
sum(train['ID'].value_counts()>1)

In [ ]:
train.shape

In [ ]:
train = train.drop_duplicates(subset='ID')

Please note that we wont be doing any validation, instead will train on the whole dataset

### Training on new word2vec model

In [ ]:
import gensim
import logging
from gensim.models import Word2Vec

wv = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/Hate_Speech_Detection_git/model_transfer_learning.txt", binary=False)
# Above is domain + pretrained embeddings

# wv = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/Hate_Speech_Detection_git/GoogleNews-vectors-negative300.bin.gz", binary=True)
# above is only pretrained embeddings

wv.init_sims(replace=True)


In [ ]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [ ]:
from sklearn.model_selection import train_test_split

def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
# train_w2v, test_w2v = train_test_split(train, test_size=0.2, random_state = 42)

train_w2v = train.copy()

# test_tokenized = test_w2v.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values
train_tokenized = train_w2v.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
# X_test_word_average = word_averaging_list(wv,test_tokenized)

In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report, roc_auc_score, confusion_matrix, accuracy_score

In [ ]:
def model_training(clf, x_t, y_t, x_v=None , y_v=None ,task='binary:logistic'):
    clf.fit(x_t,y_t)
    print('training accuracy', clf.score(x_t,y_t))
    
    #  we are not really validating anything here so lets just comment these lines out
    # if task=='binary:logistic':
    #   print('validation accuracy', clf.score(x_v,y_v))
    #   print('validation f1_score',f1_score(clf.predict(x_v),y_v))
    #   print('validation roc_auc score',roc_auc_score(y_v,clf.predict_proba(x_v)[::,-1]))
    #   print('confusion matrix \n',confusion_matrix(y_v, clf.predict(x_v)))
    
    # if task=='reg:linear':
    #   print('validation r2_score', clf.score(x_v,y_v))
    #   print('validation MSE',mean_squared_error(clf.predict(x_v),y_v))

            
    return clf

In [ ]:
%%time
xgb_w2v = XGBClassifier(n_estimators=500, max_depth=5,learning_rate=0.1,scale_pos_weight=1.4266790777602751)
lgr_w2v = LogisticRegression(n_jobs=1)

model_training(xgb_w2v,X_train_word_average,train_w2v['class'])

# model_training(lgr_w2v,X_train_word_average,train_w2v['class'],X_test_word_average,test_w2v['class'])

In [ ]:

# Logistic Regression LogisticRegression(n_jobs=1)  Domain + pretrained EMbeddings
# training accuracy 0.8715668902670548
# validation accuracy 0.8672487977727158
# validation f1_score 0.8934051417538867
# validation roc_auc score 0.9390565768416776
# confusion matrix 
#  [[2457  475]
#  [ 574 4396]]

# XGBoost Domain + Pretrained Embeddings
# training accuracy 0.9944310846728263
# validation accuracy 0.8772462667679068
# validation f1_score 0.9021190716448032
# validation roc_auc score 0.950407012333208
# confusion matrix 
#  [[2462  470]
#  [ 500 4470]]

# Logistic Regression LogisticRegression(n_jobs=1)  Only Pretrained EMbeddings
# training accuracy 0.8596696620680927
# validation accuracy 0.8535813718046065
# validation f1_score 0.8819749056411302
# validation roc_auc score 0.9265340679822454
# confusion matrix 
#  [[2422  510]
#  [ 647 4323]]
# CPU times: user 2.16 s, sys: 162 ms, total: 2.33 s
# Wall time: 2.28 s

# XGBoost + Pretrained Embeddings
# training accuracy 0.9898114162764207
# validation accuracy 0.8599088838268792
# validation f1_score 0.8885084097089335
# validation roc_auc score 0.935837432507734
# confusion matrix 
#  [[2384  548]
#  [ 559 4411]]


In [ ]:
import joblib
#save model
joblib.dump(xgb_w2v, 'xgb_deployement_trained_on_whole_dataset.pkl') 

# #load saved model
# xgb = joblib.load(filename)